# **Laboratorio 5**
### **Integrantes:**
* Maria Camila Parra Díaz (201819464)
* Esteban Emmanuel Ortiz Morales (201913613)  
* Sergio Julian Zona Moreno (201914936)

# **Depuración y análisis de los datos**

In [1]:
# Librerías para manejo de datos
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 25) # Número máximo de columnas a mostrar
pd.set_option('display.max_rows', 50) # Numero máximo de filas a mostar

from nltk import word_tokenize, sent_tokenize
import re, string, unicodedata

# Para preparar los datos
# ------------------------
# Preprocesar
# Para visualizar un reporte de los datos.
#from pandas_profiling import ProfileReport

In [2]:
# Se cargan los datos.
df_city=pd.read_csv('dimension_city.csv', sep=',', encoding = 'latin-1', index_col=None)
df_customer=pd.read_csv('dimension_customer.csv', sep=',', encoding = 'latin-1', index_col=None)
df_date=pd.read_csv('dimension_date.csv', sep=',', encoding = 'latin-1', index_col=None)
df_employee=pd.read_csv('dimension_employee.csv', sep=',', encoding = 'latin-1', index_col=None)
df_stock_item=pd.read_csv('dimension_stock_item.csv', sep=',', encoding = 'latin-1', index_col=None) 
df_fact_order=pd.read_csv('fact_order.csv', sep=',', encoding = 'latin-1', index_col=None)

# Se reemplazan los espacios de los nombres de las columnas por ''.
df_customer.columns = df_customer.columns.str.replace(' ', '')

# Dimension City

In [3]:
df_city.sample(10)

,row ID,City_Key,City,State_Province,Country,Continent,Sales_Territory,Region,Subregion,Latest_Recorded_Population
1140,Row1140,1140,Red Oak,Texas,United States,North America,Southwest,Americas,Northern America,10769
21019,Row21019,21019,Deans,New Jersey,United States,North America,Mideast,Americas,Northern America,0
115378,Row115378,115378,Vacaville,California,United States,North America,Far West,Americas,Northern America,92428
50091,Row50091,50091,Castorland,New York,United States,North America,Mideast,Americas,Northern America,351
70530,Row70530,70530,Dellview,North Carolina,United States,North America,Southeast,Americas,Northern America,13
13753,Row13753,13753,Kasilof,Alaska,United States,North America,Far West,Americas,Northern America,549
92121,Row92121,92121,Rifton,New York,United States,North America,Mideast,Americas,Northern America,456
111140,Row111140,111140,Peachburg,Alabama,United States,North America,Southeast,Americas,Northern America,0
30295,Row30295,30295,Sturgeon,Pennsylvania,United States,North America,Mideast,Americas,Northern America,1710
83974,Row83974,83974,Cairo,New York,United States,North America,Mideast,Americas,Northern America,1402


In [4]:
df_city.shape

(116295, 10)

In [5]:
df_city.drop(["row ID"], axis=1, inplace=True)
df_city = df_city[df_city.City != 'Unknown']
df_city['City'] = df_city['City'].apply(lambda x: unicodedata.normalize('NFKD', x).encode('ascii', 'ignore').decode('utf-8', 'ignore'))
df_city['City_Key'] = df_city['City_Key'].drop_duplicates()
df_city['City_Key'] = df_city['City_Key'].astype(int)

df_city_null = df_city.isnull().any(axis=1)
df_city_deleted_cols = df_city[df_city_null]

df_city = df_city.dropna()

df_city

,City_Key,City,State_Province,Country,Continent,Sales_Territory,Region,Subregion,Latest_Recorded_Population
1,1,Carrollton,New York,United States,North America,Mideast,Americas,Northern America,0
2,2,Carrollton,Virginia,United States,North America,Southeast,Americas,Northern America,4574
3,3,Carrollton,Illinois,United States,North America,Great Lakes,Americas,Northern America,2484
4,4,Carrollton,Missouri,United States,North America,Plains,Americas,Northern America,3784
5,5,Carrollton,Ohio,United States,North America,Great Lakes,Americas,Northern America,3241
...,...,...,...,...,...,...,...,...,...
116290,116290,Keyser,West Virginia,United States,North America,Southeast,Americas,Northern America,5439
116291,116291,Keystone,West Virginia,United States,North America,Southeast,Americas,Northern America,282
116292,116292,Kiahsville,West Virginia,United States,North America,Southeast,Americas,Northern America,0
116293,116293,Kimball,West Virginia,United States,North America,Southeast,Americas,Northern America,194


# Dimension Customer

In [6]:
df_customer.sample(10)

,Customer_Key,Customer,Bill_To_Customer,Category,Buying_Group,Primary_Contact,Postal_Code
199,199,Tailspin Toys (Antonito- CO),Tailspin Toys (Head Office),Novelty Shop,Tailspin Toys,Cezar Pavel,90346.0
39,39,Tailspin Toys (Diablock- KY),Tailspin Toys (Head Office),Novelty Shop,Tailspin Toys,Ashish Acharya,90158.0
205,205,Wingtip Toys (Penns Creek- PA),Wingtip Toys (Head Office),Novelty Shop,Wingtip Toys,Duleep Srivastava,90195.0
193,193,Tailspin Toys (Knifley- KY),Tailspin Toys (Head Office),Novelty Shop,Tailspin Toys,Jack Carpenter,90298.0
68,68,Tailspin Toys (Gardere- LA),Tailspin Toys (Head Office),Novelty Shop,Tailspin Toys,Valentina Conti,90136.0
160,160,Tailspin Toys (Crary- ND),Tailspin Toys (Head Office),Novelty Shop,Tailspin Toys,Bishwanath Ghosh,90248.0
301,301,Wingtip Toys (Obion- TN),Wingtip Toys (Head Office),Novelty Shop,Wingtip Toys,Bipin Modi,90451.0
351,351,Wingtip Toys (Morrison Bluff- AR),Wingtip Toys (Head Office),Novelty Shop,Wingtip Toys,Gundega Kundzina,90682.0
187,187,Tailspin Toys (Brown City- MI),Tailspin Toys (Head Office),Novelty Shop,Tailspin Toys,Biplab De,90503.0
296,296,Wingtip Toys (Boalsburg- PA),Wingtip Toys (Head Office),Novelty Shop,Wingtip Toys,Claude Bonsaint,90247.0


In [7]:
df_customer.shape

(403, 7)

In [8]:
df_customer_deleted_cols = df_customer[df_customer.Customer == 'Unknown']

df_customer = df_customer[df_customer.Customer != 'Unknown']
df_customer['Primary_Contact'] = df_customer['Primary_Contact'].apply(lambda x: unicodedata.normalize('NFKD', x).encode('ascii', 'ignore').decode('utf-8', 'ignore'))
df_customer['Customer'] = df_customer['Customer'].apply(lambda x: unicodedata.normalize('NFKD', x).encode('ascii', 'ignore').decode('utf-8', 'ignore'))
df_customer['Customer_Key'] = df_customer['Customer_Key'].drop_duplicates(keep='first')
df_customer['Postal_Code'] = df_customer['Postal_Code'].astype(int)

df_customer.dropna()

df_customer

,Customer_Key,Customer,Bill_To_Customer,Category,Buying_Group,Primary_Contact,Postal_Code
1,1,Tailspin Toys (Head Office),Tailspin Toys (Head Office),Novelty Shop,Tailspin Toys,Waldemar Fisar,90410
2,2,Tailspin Toys (Sylvanite- MT),Tailspin Toys (Head Office),Novelty Shop,Tailspin Toys,Lorena Cindric,90216
3,3,Tailspin Toys (Peeples Valley- AZ),Tailspin Toys (Head Office),Novelty Shop,Tailspin Toys,Bhaargav Rambhatla,90205
4,4,Tailspin Toys (Medicine Lodge- KS),Tailspin Toys (Head Office),Novelty Shop,Tailspin Toys,Daniel Roman,90152
5,5,Tailspin Toys (Gasport- NY),Tailspin Toys (Head Office),Novelty Shop,Tailspin Toys,Johanna Huiting,90261
...,...,...,...,...,...,...,...
398,398,Wingtip Toys (Haycock- AK),Wingtip Toys (Head Office),Novelty Shop,Wingtip Toys,Bhaamini Kanaparthi,90603
399,399,Wingtip Toys (Kapa'a- HI),Wingtip Toys (Head Office),Novelty Shop,Wingtip Toys,Margit Molnar,90796
400,400,Wingtip Toys (Dickworsham- TX),Wingtip Toys (Head Office),Novelty Shop,Wingtip Toys,Chandni Jandhyala,90704
401,401,Wingtip Toys (Caro- MI),Wingtip Toys (Head Office),Novelty Shop,Wingtip Toys,Ljubomir Nesic,90043


# Dimension Employee

In [9]:
df_employee.sample(10)

,Employee_Key,Employee,Preferred_Name,Is_Salesperson
26,26,Taj Shand,Taj,True
136,136,Jack Potter,Jack,True
81,81,Kayla Woodcock,Kayla,True
151,151,Hudson Hollinworth,Hudson,True
139,139,Alica Fatnowna,Alica,False
12,12,Sophia Hinton,Sophia,True
161,161,Sophia Hinton,Sophia,True
90,90,Hudson Hollinworth,Hudson,True
6,6,Anthony Grosse,Anthony,True
185,185,Stella Rosenhain,Stella,False


In [10]:
df_employee.shape

(213, 4)

In [11]:
df_employee_deleted_cols = df_employee[df_employee.Employee == 'Unknown']

df_employee = df_employee[df_employee.Employee != 'Unknown']
df_employee['Employee'] = df_employee['Employee'].apply(lambda x: unicodedata.normalize('NFKD', x).encode('ascii', 'ignore').decode('utf-8', 'ignore'))
df_employee['Preferred_Name'] = df_employee['Preferred_Name'].apply(lambda x: unicodedata.normalize('NFKD', x).encode('ascii', 'ignore').decode('utf-8', 'ignore'))

df_employee = df_employee.dropna()

df_employee

,Employee_Key,Employee,Preferred_Name,Is_Salesperson
1,1,Lily Code,Lily,True
2,2,Isabella Rupp,Isabella,False
3,3,Ethan Onslow,Ethan,False
4,4,Amy Trefl,Amy,True
5,5,Jai Shand,Jai,False
...,...,...,...,...
208,208,Archer Lamble,Archer,True
209,209,Piper Koch,Piper,False
210,210,Katie Darwin,Katie,False
211,211,Jai Shand,Jai,False


# Dimension Stock Item

In [12]:
df_stock_item.sample(10)

,Stock_Item_Key,Stock_Item,Color,Selling_Package,Buying_Package,Brand,Size_val,Lead_Time_Days,Quantity_Per_Outer,Is_Chiller_Stock,Tax_Rate,Unit_Price,Recommended_Retail_Price,Typical_Weight_Per_Unit
199,199,DBA joke mug - you might be a DBA if (Black),Black,Each,Each,NaN,NaN,12,1,False,"12,000","13,00","19,44",",150"
636,636,Shipping carton (Brown) 305x305x305mm,NaN,Each,Each,NaN,305x305x305mm,14,25,False,"14,000","3,50","5,23",",300"
504,504,Furry gorilla with big eyes slippers (Black) S,Black,Each,Each,NaN,S,12,1,False,"12,000","32,00","47,84",",350"
667,667,Air cushion machine (Blue),NaN,Each,Each,NaN,NaN,20,1,False,"20,000","1899,00","2839,01","10,000"
133,133,The Gu red shirt XML tag t-shirt (White) 6XL,White,Each,Carton,NaN,6XL,7,12,False,"7,000","18,00","26,91",",400"
293,293,Void fill 400 L bag (White) 400L,NaN,Each,Each,NaN,400L,14,10,False,"14,000","50,00","74,75","1,000"
36,36,Shipping carton (Brown) 305x305x305mm,NaN,Each,Each,NaN,305x305x305mm,14,25,False,"14,000","3,50","5,23",",300"
419,419,The Gu red shirt XML tag t-shirt (Black) XXL,Black,Each,Carton,NaN,XXL,7,12,False,"7,000","18,00","26,91",",400"
363,363,Animal with big feet slippers (Brown) S,NaN,Each,Each,NaN,S,12,1,False,"12,000","32,00","47,84",",350"
473,473,RC vintage American toy coupe with remote cont...,Red,Each,Each,Northwind,1/50 scale,14,1,False,"14,000","30,00","44,85","1,700"


In [13]:
df_stock_item.shape

(672, 14)

In [14]:
df_stock_item = df_stock_item[df_stock_item.Stock_Item != 'Unknown']
df_stock_item['Stock_Item'] = df_stock_item['Stock_Item'].apply(lambda x: unicodedata.normalize('NFKD', x).encode('ascii', 'ignore').decode('utf-8', 'ignore'))

df_stock_item["Color"].fillna("NAN", inplace=True)
df_stock_item["Size_val"].fillna("NAN", inplace=True)
df_stock_item.drop(["Brand"], axis=1, inplace=True)

df_stock_item['Tax_Rate'] = [x.replace(',','.') for x in df_stock_item['Tax_Rate']]
df_stock_item['Unit_Price'] = [x.replace(',','.') for x in df_stock_item['Unit_Price']]
df_stock_item['Recommended_Retail_Price'] = [x.replace(',','.') for x in df_stock_item['Recommended_Retail_Price']]
df_stock_item['Typical_Weight_Per_Unit'] = [x.replace(',','.') for x in df_stock_item['Typical_Weight_Per_Unit']]

df_stock_item_null = df_stock_item.isnull().any(axis=1)
df_stock_item_deleted_cols = df_stock_item[df_stock_item_null]
df_stock_item = df_stock_item.dropna()

df_stock_item

,Stock_Item_Key,Stock_Item,Color,Selling_Package,Buying_Package,Size_val,Lead_Time_Days,Quantity_Per_Outer,Is_Chiller_Stock,Tax_Rate,Unit_Price,Recommended_Retail_Price,Typical_Weight_Per_Unit
1,1,Void fill 400 L bag (White) 400L,NAN,Each,Each,400L,14,10,False,14.000,50.00,74.75,1.000
2,2,Void fill 300 L bag (White) 300L,NAN,Each,Each,300L,14,10,False,14.000,37.50,56.06,.750
3,3,Void fill 200 L bag (White) 200L,NAN,Each,Each,200L,14,10,False,14.000,25.00,37.38,.500
4,4,Void fill 100 L bag (White) 100L,NAN,Each,Each,100L,14,10,False,14.000,12.50,18.69,.250
5,5,Air cushion machine (Blue),NAN,Each,Each,NAN,20,1,False,20.000,1899.00,2839.01,10.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,667,Air cushion machine (Blue),NAN,Each,Each,NAN,20,1,False,20.000,1899.00,2839.01,10.000
668,668,Void fill 100 L bag (White) 100L,NAN,Each,Each,100L,14,10,False,14.000,12.50,18.69,.250
669,669,Void fill 200 L bag (White) 200L,NAN,Each,Each,200L,14,10,False,14.000,25.00,37.38,.500
670,670,Void fill 300 L bag (White) 300L,NAN,Each,Each,300L,14,10,False,14.000,37.50,56.06,.750


# Dimension Fact Order

In [15]:
df_fact_order.sample(10)

,Order_Key,City_Key,Customer_Key,Stock_Item_Key,Order_Date_Key,Picked_Date_Key,Salesperson_Key,Picker_Key,Package,Quantity,Unit_Price,Tax_Rate,Total_Excluding_Tax,Tax_Amount,Total_Including_Tax
206834,206835,94333,0,47,2016-01-27,2016-01-27,151,155,Each,90,"105,00","15,000","9450,00","1417,50","10867,50"
104141,104142,100428,308,154,2014-08-19,2014-08-19,118,108,Each,1,"45,00","15,000","45,00","6,75","51,75"
166867,166868,76043,267,44,2015-07-10,2015-07-10,155,142,Each,3,"240,00","15,000","720,00","108,00","828,00"
79852,79853,91372,201,102,2014-04-15,2014-04-15,102,81,Each,8,"32,00","15,000","256,00","38,40","294,40"
215731,215732,107284,0,177,2016-03-17,2016-03-17,163,166,Each,5,"13,00","15,000","65,00","9,75","74,75"
114952,114953,93314,0,63,2014-10-20,2014-10-20,108,124,Each,10,"30,00","15,000","300,00","45,00","345,00"
174213,174214,47471,0,164,2015-08-14,2015-08-14,151,154,Each,10,"13,00","15,000","130,00","19,50","149,50"
26677,26678,48331,347,190,2013-06-07,2013-06-07,65,71,Each,6,"13,00","15,000","78,00","11,70","89,70"
97360,97361,92524,0,167,2014-07-14,2014-07-14,93,93,Each,5,"13,00","15,000","65,00","9,75","74,75"
77825,77826,41398,52,202,2014-04-03,2014-04-03,86,87,Each,6,"13,00","15,000","78,00","11,70","89,70"


In [16]:
df_fact_order.shape

(231412, 15)

In [17]:
df_fact_order_null = df_fact_order.isnull().any(axis=1)
df_fact_order_deleted_cols = df_fact_order[df_fact_order_null]

df_fact_order = df_fact_order.dropna()
print(df_fact_order.shape)

# eliminación de llaves foraneas de objetos de no existen
df_fact_order_city = ~df_fact_order["City_Key"].isin(df_city_deleted_cols["City_Key"])
df_fact_order = df_fact_order[df_fact_order_city]
print(df_fact_order.shape)

df_fact_order_customer = ~df_fact_order["Customer_Key"].isin(df_customer_deleted_cols["Customer_Key"])
df_fact_order = df_fact_order[df_fact_order_customer]
print(df_fact_order.shape)

df_fact_order_employee = ~df_fact_order["Salesperson_Key"].isin(df_employee_deleted_cols["Employee_Key"])
df_fact_order = df_fact_order[df_fact_order_employee]
print(df_fact_order.shape)

df_fact_order_picker = ~df_fact_order["Picker_Key"].isin(df_employee_deleted_cols["Employee_Key"])
df_fact_order = df_fact_order[df_fact_order_picker]
print(df_fact_order.shape)

df_fact_order_stock_item = ~df_fact_order["Stock_Item_Key"].isin(df_stock_item_deleted_cols["Stock_Item_Key"])
df_fact_order = df_fact_order[df_fact_order_stock_item]
print(df_fact_order.shape)



df_fact_order['Unit_Price'] = [x.replace(',','.') for x in df_fact_order['Unit_Price']]
df_fact_order['Tax_Rate'] = [x.replace(',','.') for x in df_fact_order['Tax_Rate']]
df_fact_order['Total_Excluding_Tax'] = [x.replace(',','.') for x in df_fact_order['Total_Excluding_Tax']]
df_fact_order['Tax_Amount'] = [x.replace(',','.') for x in df_fact_order['Tax_Amount']]
df_fact_order['Total_Including_Tax'] = [x.replace(',','.') for x in df_fact_order['Total_Including_Tax']]

(228265, 15)
(228265, 15)
(143968, 15)
(143968, 15)
(130901, 15)
(130901, 15)


In [18]:
df_fact_order

,Order_Key,City_Key,Customer_Key,Stock_Item_Key,Order_Date_Key,Picked_Date_Key,Salesperson_Key,Picker_Key,Package,Quantity,Unit_Price,Tax_Rate,Total_Excluding_Tax,Tax_Amount,Total_Including_Tax
4,5,70644,57,14,2013-01-01,2013-01-01,19,11,Each,96,2.70,15.000,259.20,38.88,298.08
5,6,70644,57,90,2013-01-01,2013-01-01,19,11,Each,5,32.00,15.000,160.00,24.00,184.00
6,7,70644,57,170,2013-01-01,2013-01-01,19,11,Each,2,13.00,15.000,26.00,3.90,29.90
37,38,69146,224,58,2013-01-01,2013-01-02,15,20,Each,100,22.00,15.000,2200.00,330.00,2530.00
38,39,69146,224,180,2013-01-01,2013-01-02,15,20,Each,10,13.00,15.000,130.00,19.50,149.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231397,231398,99099,205,161,2016-05-31,2016-05-31,168,142,Each,6,25.00,15.000,150.00,22.50,172.50
231398,231399,99099,205,218,2016-05-31,2016-05-31,168,142,Each,1,25.00,15.000,25.00,3.75,28.75
231399,231400,99099,205,118,2016-05-31,2016-05-31,168,142,Each,9,35.00,15.000,315.00,47.25,362.25
231400,231401,99099,205,50,2016-05-31,2016-05-31,168,142,Each,40,102.00,15.000,4080.00,612.00,4692.00


In [19]:
df_city.to_csv('dimension_city.csv', encoding="utf-8", index=False)
df_customer.to_csv('dimension_customer.csv', encoding="utf-8", index=False)
df_date.to_csv('dimension_date.csv', encoding="utf-8", index=False)
df_employee.to_csv('dimension_employee.csv', encoding="utf-8", index=False)
df_stock_item.to_csv('dimension_stock_item.csv', encoding="utf-8", index=False)
df_fact_order.to_csv('fact_order.csv', encoding="utf-8", index=False)